In [11]:
import argparse
import glob
import numpy as np
import os 
from tqdm import tqdm
import matplotlib.pyplot as plt
from tabulate import tabulate
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
def calc_min_max(x):
    # x = (x.astype(np.float32)/127.5-1)
    return np.nanmin(x,axis=1),np.nanmax(x,axis=1)

In [24]:
src = r"..\repos\Palette\experiments\train_BPv8_complete_6"
epoch = 53 
epoch = str(epoch)

In [37]:
scales.dtype
val = np.load(os.path.join(r"..\repos\Palette",scale_dir,"validate.npy"))
val.shape,val.dtype

((44866, 256, 2), dtype('float64'))

In [29]:
with open(os.path.join(src,"config.json"), "r") as file:
    json_data = json.load(file)

scale_dir = json_data["datasets"]["train"]["which_dataset"]["args"]["data_root"]
print(scale_dir)

# load scale file
scales = np.load(os.path.join(r"..\repos\Palette",scale_dir,"scale_train.npy"))

result_dir = os.path.join(src,"results","test",epoch)
gt = []
cond= []
out = []
gt_files = glob.glob(f"{result_dir}\GT_*.npy")
cond_files = glob.glob(f"{result_dir}\Process_*.npy")
out_files = glob.glob(f"{result_dir}\OUT_*.npy")
for j in tqdm(range(len(gt_files))):
    gt.append(np.load(os.path.join(gt_files[j])))
    cond.append(np.load(os.path.join(cond_files[j])))
    out.append(np.load(os.path.join(out_files[j])))
print(gt[-1].shape)
out1 = np.concatenate(out,axis=0)
gt1 = np.concatenate(gt,axis=0)
cond1 = np.concatenate(cond,axis=0)
out1 = out1.astype(np.float32)
gt1 = gt1.astype(np.float32)
cond1 = cond1.astype(np.float32)
scales = scales[:out1.shape[0]]
print(scales.shape,gt1.shape,out1.shape,scales)
# 正規化を元に戻す
gt1[:] -= scales[:,0][:, None]
gt1[:] /= scales[:,1][:, None]
out1[:] -= scales[:,0][:, None]
out1[:] /= scales[:,1][:, None]

..\..\data\processed\BP_npy\0325_256_corr\p00


  0%|          | 0/173 [00:00<?, ?it/s]

100%|██████████| 173/173 [00:00<00:00, 1820.99it/s]

(500, 256)


(1, 2) (86272, 256) (86272, 256) [[-0.38464342  0.00769287]]


In [30]:
out2 = np.concatenate(out,axis=0)
gt2 = np.concatenate(gt,axis=0)
out2 =out2.astype(np.float32)
gt2 = gt2.astype(np.float32)

print(out2.shape,gt2.shape)
scaler = MinMaxScaler()
scaler.min_ = scales[0,0]
scaler.scale_ = scales[0,1]
out2 = scaler.inverse_transform(out2.reshape(1, -1) ).reshape(-1,256)
gt2 = scaler.inverse_transform(gt2.reshape(1, -1) ).reshape(-1,256)

(86272, 256) (86272, 256)


In [31]:
out2.dtype,out[0].dtype,gt[0].dtype

(dtype('float32'), dtype('float16'), dtype('float16'))

In [32]:
mean_absolute_error(out1.flatten(),out2.flatten())

9.0270834e-07

In [35]:
gt_mean = np.mean(gt1.flatten())
out_mean = np.mean(out1.flatten())
cond_mean = np.mean(cond1.flatten())
gt_std = np.std(gt1.mean(axis=1),dtype=np.float64)
out_std = np.std(out1.mean(axis=1),dtype=np.float64)
cond_std = np.std(cond1.mean(axis=1),dtype=np.float64)
print(np.count_nonzero(np.isnan(out1)))

headers = ["Signal", "Mean","Std"]
table=[]
table.append(["cond_ppg",cond_mean,cond_std])
# table.append(["data_ppg", 0.494153162946643,0.10694360087091538])
# table.append(["data_abp",0.39593121533751857,0.13489903083932583])
table.append(["gt",gt_mean,gt_std])
table.append(["out",out_mean,out_std])
print(tabulate(table,headers, floatfmt=".4f"))
# 血圧の最大値(SBP)最小値(DBP)を計算
gt_min,gt_max = calc_min_max(gt1)
out_min,out_max = calc_min_max(out1)
# 誤差
errors = np.zeros((2,*out_min.shape))
errors[0,:]=gt_min-out_min
errors[1,:]=gt_max-out_max
errors = errors[:,~np.isnan(errors).any(axis=0)]
print(np.count_nonzero(np.isnan(errors)))
print("error shape:",errors.shape)
n_samples = errors.shape[1]
print("n_samples:",n_samples)
me = np.mean(errors,axis=1)
mae = np.mean(np.abs(errors),axis=1)
rmse = np.sqrt(np.mean(errors**2,axis=1))
std = np.std(errors,axis=1)
print(me.shape,mae.shape,rmse.shape,std.shape)
error_5 = np.count_nonzero(np.abs(errors)<=5,axis=1)/n_samples*100
error_15 = np.count_nonzero(np.abs(errors)<=15,axis=1)/n_samples*100
error_10 = np.count_nonzero(np.abs(errors)<=10,axis=1)/n_samples*100
print("""
        test data samples:
        # samples : {}
        
        Eval Stats:   SBP    DBP
        MAE:        {:6.3f} {:6.3f}
        RMSE:       {:6.3f} {:6.3f}
        Mean Error: {:6.3f} {:6.3f}
        STD:        {:6.3f} {:6.3f}
        
        BHS standards range:
        Error   <5mmHg <10mmHg <15mmHg
        gradeA     60%     85%     95%
        gradeB     50%     75%     90%
        gradeC     40%     65%     85%
        SBP     {:5.1f}%  {:5.1f}%  {:5.1f}%
        DBP     {:5.1f}%  {:5.1f}%  {:5.1f}%
        
        
        """.format(
        n_samples,
        *mae,
        *rmse,
        *me,
        *std,
        error_5[0], error_10[0], error_15[0],
        error_5[1], error_10[1], error_15[1],
        ))

0
Signal       Mean      Std
--------  -------  -------
cond_ppg   0.4231   0.0132
gt        90.2553  12.7645
out       94.3540  13.1323
0
error shape: (2, 86272)
n_samples: 86272
(2,) (2,) (2,) (2,)

        test data samples:
        # samples : 86272
        
        Eval Stats:   SBP    DBP
        MAE:         9.707 20.687
        RMSE:       12.692 25.669
        Mean Error: -2.636 -7.879
        STD:        12.415 24.430
        
        BHS standards range:
        Error   <5mmHg <10mmHg <15mmHg
        gradeA     60%     85%     95%
        gradeB     50%     75%     90%
        gradeC     40%     65%     85%
        SBP      35.1%   61.8%   78.3%
        DBP      15.2%   29.6%   43.2%
        
        
        


In [38]:
data_root = r"../data/processed/BP_npy/0325_256_corr/p00"
val = np.load(os.path.join(data_root,"validate.npy"))
test = np.load(os.path.join(data_root,"test.npy"))
scale = np.load(os.path.join(data_root,"scale_train.npy"))
print(val.shape,test.shape)
scaler = MinMaxScaler()
scaler.min_ = scales[0,0]
scaler.scale_ = scales[0,1]
val_ppg= scaler.inverse_transform(out2.reshape(1, -1) ).reshape(-1,256)
gt2 = scaler.inverse_transform(gt2.reshape(1, -1) ).reshape(-1,256)


(44866, 256, 2) (86304, 256, 2)
